## Setups & Imports

In [2]:
# imports
import asf_search as asf
import credentials_asf
import geopandas as gpd
import os
import pandas as pd
from datetime import date
from shapely.geometry import box
from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 20)

In [ ]:
# define relevant paths
work_dir = "C://Users/felix/github/sar_parcel_analyses"
download_dir = "D://Adv_Rs_project/01_data_raw/s1_slc"

## Reading AOI & Downloading Scenes

In [3]:
# read aoi shapefile and transform its bbox to wkt
aoi = gpd.read_file(os.path.join(work_dir, "01_data_raw", "fields.geojson"))
aoi_bounds = aoi.total_bounds 
aoi_bounds = gpd.GeoSeries([box(*aoi_bounds)])
aoi_bounds = aoi_bounds.to_wkt().values.tolist()[0]

# write wkt to file
with open(os.path.join(work_dir, "01_data_raw", "fields.txt"), "wt") as f:
    f.write(aoi_bounds)

In [3]:
# search for S1 SLC data via ASF interface
results = asf.search(
    platform = asf.PLATFORM.SENTINEL1,
    processingLevel=[asf.PRODUCT_TYPE.SLC],
    start = date(2021, 4, 1),
    end = date(2021, 10, 31),
    intersectsWith = aoi_bounds
    )
    
print(f'Total Images Found: {len(results)}')

Total Images Found: 205


In [4]:
# evaluate metadata of found scenes
metadata = pd.DataFrame([x.properties for x in results])
metadata = metadata[["platform", "polarization", "flightDirection", "pathNumber", "frameNumber", "startTime", "centerLat", "centerLon", "fileName", "url"]]
metadata["startTime"] = pd.to_datetime(metadata["startTime"], format='%Y-%m-%dT%H:%M:%S')
metadata.describe(include="all", datetime_is_numeric=True)

,platform,polarization,flightDirection,pathNumber,frameNumber,startTime,centerLat,centerLon,fileName,url
count,205,205,205,205,205,205,205,205,205,205
unique,2,1,2,4,8,NaN,124,154,205,205
top,Sentinel-1B,VV+VH,DESCENDING,22,430,NaN,47.5772,13.2196,S1A_IW_SLC__1SDV_20211030T051039_20211030T0511...,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...
freq,116,205,117,68,64,NaN,5,5,1,1
mean,NaN,NaN,NaN,NaN,NaN,2021-07-20 12:35:21.619512064,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,2021-04-02 05:18:15,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,2021-06-01 05:18:18,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,2021-07-21 16:58:34,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,2021-09-11 05:17:42,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,2021-10-30 05:10:39,NaN,NaN,NaN,NaN


In [5]:
# filter scenes based on relative orbit
scene_selection = metadata[metadata["pathNumber"].isin(["44", "95"])]
scene_selection = scene_selection[scene_selection["frameNumber"].isin(["430", "432", "151", "152"])]
scene_selection.sort_values(["startTime"])

,platform,polarization,flightDirection,pathNumber,frameNumber,startTime,centerLat,centerLon,fileName,url
204,Sentinel-1A,VV+VH,DESCENDING,95,430,2021-04-02 05:18:15,48.6521,13.2205,S1A_IW_SLC__1SDV_20210402T051815_20210402T0518...,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...
202,Sentinel-1B,VV+VH,ASCENDING,44,151,2021-04-04 16:58:29,47.577,13.0895,S1B_IW_SLC__1SDV_20210404T165829_20210404T1658...,https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_...
197,Sentinel-1A,VV+VH,ASCENDING,44,152,2021-04-10 16:59:13,47.6645,13.0773,S1A_IW_SLC__1SDV_20210410T165913_20210410T1659...,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...
195,Sentinel-1A,VV+VH,DESCENDING,95,430,2021-04-14 05:18:16,48.6522,13.2196,S1A_IW_SLC__1SDV_20210414T051816_20210414T0518...,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...
193,Sentinel-1B,VV+VH,ASCENDING,44,151,2021-04-16 16:58:29,47.5771,13.0893,S1B_IW_SLC__1SDV_20210416T165829_20210416T1658...,https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_...
...,...,...,...,...,...,...,...,...,...,...
15,Sentinel-1B,VV+VH,DESCENDING,95,430,2021-10-17 05:17:43,48.6759,13.2363,S1B_IW_SLC__1SDV_20211017T051743_20211017T0518...,https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_...
11,Sentinel-1A,VV+VH,ASCENDING,44,152,2021-10-19 16:59:22,47.665,13.0773,S1A_IW_SLC__1SDV_20211019T165922_20211019T1659...,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...
9,Sentinel-1A,VV+VH,DESCENDING,95,430,2021-10-23 05:18:25,48.6526,13.2196,S1A_IW_SLC__1SDV_20211023T051825_20211023T0518...,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...
6,Sentinel-1B,VV+VH,ASCENDING,44,151,2021-10-25 16:58:38,47.5776,13.0876,S1B_IW_SLC__1SDV_20211025T165838_20211025T1659...,https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_...


In [11]:
# download S1 data
# get urls & split into batches for parallel download
urls = [x for x in scene_selection.sort_values(["startTime"]).iloc[:,]["url"]]
url_batches = [urls[i:i + 4] for i in range(0, len(urls), 4)]

['https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20210402T051815_20210402T051843_037267_0463E1_4FA2.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210404T165829_20210404T165857_026320_032437_1DA7.zip',
 'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20210410T165913_20210410T165940_037391_046811_38EF.zip',
 'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20210414T051816_20210414T051844_037442_0469EF_5AA1.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210416T165829_20210416T165857_026495_0329CA_9CB3.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210420T051746_20210420T051813_026546_032B84_2810.zip',
 'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20210422T165913_20210422T165940_037566_046E25_93D0.zip',
 'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20210426T051816_20210426T051844_037617_046FF7_C8D7.zip',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210428T165830_202104

In [10]:
# authenticate session
session = asf.ASFSession().auth_with_creds(credentials_asf.username, credentials_asf.password)

# download files
for url_batch in tqdm(url_batches, desc='downloading S1 files'):
    asf.download_urls(
        urls=url_batch,
        path=os.path.join(download_dir), 
        session=session,
        processes=4)

# alternative way to download queried results directly
# results.download(
#      path = os.path.join(work_dir, "01_data", "s1_slc"),
#      session = session, 
#      processes = 8
#   )

downloading S1 files:   0%|          | 0/1 [00:00<?, ?it/s]